# 01. Importamos las librerias

In [2]:
import pandas as pd # Cargamos la libreria de "pandas" para la manipulación y el análisis de datos
import numpy as np # Cargamos la librería de "numpy" para realizar cálculos lógicos y matemáticos sobre cuadros y matrices en el caso que lo necesitemos
import pyarrow #Cargando la librería "json" nos permitira manipular archivos tipo parquet
import re
import os

# 02. Leemos los datasets para el Modelado del Sistema de Recomendación

In [3]:
# Leerlos desde local
path = 'data/recommend.parquet'
df_recommend = pd.read_parquet(path)

# 03. Seleccionar un Estado

In [8]:
Lista_Estados = df_recommend.state_name.unique()
print('Estados a Seleccionar: ')
print("_____________________________")
print(Lista_Estados)

Estados a Seleccionar: 
_____________________________
['Alabama' 'Louisiana' 'Mississippi' 'South Carolina' 'Wisconsin'
 'Maryland' 'North Carolina' 'South Dakota' 'Texas' 'Kansas' 'Illinois'
 'Virginia' 'Massachusetts' 'Pennsylvania' 'New Jersey' 'California'
 'Maine' 'Georgia' 'Ohio' 'Oklahoma' 'Kentucky' 'New York' 'Tennessee'
 'Michigan' 'Minnesota' 'Missouri' 'Wyoming' 'Arizona' 'Hawaii' 'Iowa'
 'Nebraska' 'Colorado' 'Indiana' 'Florida' 'New Mexico' 'Oregon' 'Montana'
 'New Hampshire' 'Arkansas' 'Nevada' 'Idaho' 'Utah' 'Washington' 'Alaska'
 'Connecticut' 'West Virginia' 'Vermont' 'Rhode Island' 'Delaware'
 'North Dakota' '0' 'District of Columbia']


In [9]:
# Convert it to a DataFrame
df_lista_estados = pd.DataFrame(Lista_Estados, columns=['State'])

# Save the DataFrame to a CSV file
df_lista_estados.to_csv('data/lista_de_estados.csv', index=False)

# 04. Seleccionar un County

In [20]:
# Specify the state for which you want to list unique county values
selected_state = 'California'  # Replace 'Your_State_Name_Here' with the desired state name

# Filter the DataFrame for the selected state
state_filtered_df = df_recommend[df_recommend['state_name'] == selected_state]

# Get the unique county values for the selected state
unique_county_values = state_filtered_df['county'].unique()

# Display the unique county values for the selected state
print("Seleccionar un County de ", selected_state, ":")
print("__________________________________________________")
print(unique_county_values)

Seleccionar un County de  California :
__________________________________________________
['San Joaquin' 'San Bernardino' 'Los Angeles' 'Alameda' 'Contra Costa'
 'Orange' 'San Diego' 'Modoc' 'Napa' 'Shasta' 'Calaveras' 'Santa Cruz'
 'Colusa' 'Humboldt' 'Kings' 'Monterey' 'San Luis Obispo' 'Kern' 'Merced'
 'Placer' 'Fresno' 'Riverside' 'San Mateo' 'Solano' 'Tuolumne' 'Inyo'
 'Sonoma' 'Mendocino' 'Imperial' 'Yuba' 'Santa Barbara' 'Ventura'
 'El Dorado' 'Santa Clara' 'Sacramento' 'Stanislaus' 'Plumas' 'Butte'
 'Madera' 'Lake' 'Tehama' 'Mariposa' 'Del Norte' 'Yolo' 'Tulare' 'Marin'
 'Nevada' 'Siskiyou' 'San Benito' 'Amador' 'Lassen' 'Sutter' 'Glenn'
 'San Francisco' 'Allegheny' 'Trinity']


# 05. Recomendar una lista de 5 ciudades donde convenga invertir segun cantidad de tiendas por densidad poblacional y GDP

Nos proponemos crear una función que, aplicando filtrado interactivo y colaborativo, proporcionará una lista de las 5 ciudades del Estado y Condado seleccionado.
<br>
En esta primera función, la entrada es el Estado y luego para ese Estado,el Condado, que es el que se analizara.

In [13]:
def recomendar_locacion(state_name,county):
    '''
    Al darle como Input un Estado, se crea una matriz unica de county a seleccionar de ese estado seleccionado.
    Con el dato de County calcula las ciudades de ese county que menor presencia tiene Walgreens y menos saturado el mercado por cantidad de habitantes de la ciudad
    Genera un output de 5 recomendaciones por Condado, de las 5 mejores ciudades donde instalar una tienda Walgreens
    '''
    
    # For that State, select the available county
    selected_state = state_name
    selected_county = county

    # Leerlos desde local
    path = 'data/recommend.parquet'
    df_recommend = pd.read_parquet(path)
    # Filter the DataFrame for the selected state and county
    county_filtered_df = df_recommend[(df_recommend['state_name'] == selected_state) & (df_recommend['county'] == selected_county)]

    # Get the unique city values for the selected state and county
    unique_city_values = county_filtered_df['city'].unique()

    # Display the unique city values for the selected state and county
    # print("Unique city values for", selected_county, "County,", selected_state, ":", unique_city_values)

    # Sort the DataFrame by 'ratio_stores' column in ascending order
    sorted_county_filtered_df = county_filtered_df.sort_values(by='ratio_stores', ascending=True)

    # Calculate the total number of stores
    sorted_county_filtered_df['Total_Number_Stores'] = sorted_county_filtered_df['total_others_stores'] + sorted_county_filtered_df['total_walgreens_stores']

    # Calculate Walgreens presence
    sorted_county_filtered_df['Walgreens_Presence_%'] = round((sorted_county_filtered_df['total_walgreens_stores'] / sorted_county_filtered_df['Total_Number_Stores']) * 100, 2)

    sorted_county_filtered_df_display = sorted_county_filtered_df.drop(['city_state_county', 'county', 'state', 'state_name','others_total_reviews', 'others_average_rating', 'total_others_stores',
        'walgreens_average_rating','walgreens_total_reviews', 'population', 'density', 'ratio_stores'],axis=1) 
    sorted_county_filtered_df_display.rename(columns={'total_walgreens_stores': 'Total_Walgreens_Stores'},inplace=True)
    
    new_order = [ 'city','Total_Number_Stores','Total_Walgreens_Stores','Walgreens_Presence_%']
    sorted_county_filtered_df_display = sorted_county_filtered_df_display[new_order]
    print('Las Top 5 Ciudades recomendadas donde emplazar nuevas Tiendas de Walgreens')
    print('para el Condado de ',county,' en el Estado de ', state_name, 'son: ') 
    print('_____________________________________________________________________')
    # Display the DataFrame with the new columns
    return sorted_county_filtered_df_display.head()

In [14]:
# input
selected_state = 'California'
selected_county = 'San Diego'

In [15]:
recomendar_locacion(selected_state,selected_county)

Las Top 5 Ciudades recomendadas donde emplazar nuevas Tiendas de Walgreens
para el Condado de  San Diego  en el Estado de  California son: 
_____________________________________________________________________


,city,Total_Number_Stores,Total_Walgreens_Stores,Walgreens_Presence_%
9491,San Diego,129.0,2.0,1.55
8786,Poway,4.0,1.0,25.00
9509,San Marcos,8.0,0.0,0.00
2409,Coronado,2.0,0.0,0.00
7958,Oceanside,15.0,0.0,0.00


In [16]:
# input
selected_state = 'Florida'
selected_county = 'Miami-Dade'
recomendar_locacion(selected_state,selected_county)

Las Top 5 Ciudades recomendadas donde emplazar nuevas Tiendas de Walgreens
para el Condado de  Miami-Dade  en el Estado de  Florida son: 
_____________________________________________________________________


,city,Total_Number_Stores,Total_Walgreens_Stores,Walgreens_Presence_%
6893,Miami,153.0,20.0,13.07
6890,Miami Lakes,2.0,0.0,0.00
5542,Key Biscayne,1.0,0.0,0.00
6889,Miami Gardens,9.0,0.0,0.00
10037,South Miami,1.0,0.0,0.00


In [18]:
# input
selected_state = 'Texas'
selected_county = 'Dallas'
recomendar_locacion(selected_state,selected_county)

Las Top 5 Ciudades recomendadas donde emplazar nuevas Tiendas de Walgreens
para el Condado de  Dallas  en el Estado de  Texas son: 
_____________________________________________________________________


,city,Total_Number_Stores,Total_Walgreens_Stores,Walgreens_Presence_%
1668,Carrollton,4.0,0.0,0.00
2621,Dallas,211.0,16.0,7.58
10997,University Park,1.0,0.0,0.00
4274,Grand Prairie,11.0,1.0,9.09
9367,Rowlett,4.0,0.0,0.00
